In [1]:
from datasets import load_dataset
ds = load_dataset("merve/vqav2-small",split="validation[:10%]")
ds, ds[0]

Dataset({
    features: ['multiple_choice_answer', 'question', 'image'],
    num_rows: 2144
})

In [1]:
import os
os.environ['HF_HOME'] = '/home/sa5u24/VQA'
hf_home = os.path.expanduser(
    os.getenv("HF_HOME", os.path.join(os.getenv("XDG_CACHE_HOME", "~/.cache"), "huggingface"))
)
print(hf_home)


/home/sa5u24/VQA


In [2]:
# note the image is not provided in the prompt its included as part of the "processor"

# prompt= """Create a Short Product description based on the provided ##PRODUCT NAME## and ##CATEGORY## and image.
# Only return description. The description should be SEO optimized and for a better mobile search experience.

# ##PRODUCT NAME##: {product_name}
# ##CATEGORY##: {category}"""

prompt= """Answer the question based on the provided ##Question## and image. ##Question##: {question}"""

from datasets import load_dataset

# Convert dataset to OAI messages
def format_data(sample):
    return {"messages": [
                {
                    "role": "question",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt.format(question=sample["question"]),
                        },{
                            "type": "image",
                            "image": sample["image"],
                        }
                    ],
                },
                {
                    "role": "answer",
                    "content": [{"type": "text", "text": sample["multiple_choice_answer"]}],
                },
            ],
        }

# Load dataset from the hub
ds = load_dataset("merve/vqav2-small",split="validation[:1%]")
# Convert dataset to OAI messages
# need to use list comprehension to keep Pil.Image type, .mape convert image to bytes
dataset = [format_data(sample) for sample in ds]
dataset[0]["messages"], len(dataset)

Load model directly from the llama, not the automodel for all models

In [4]:
from transformers import MllamaForConditionalGeneration, AutoProcessor
from transformers import MllamaConfig
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model
from peft import PeftModel

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

processor = AutoProcessor.from_pretrained(model_id)
model = MllamaForConditionalGeneration.from_pretrained(
            model_id, torch_dtype=torch.bfloat16, device_map="cuda",
            quantization_config=bnb_config
        )

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

# adding lora layers in the model, with random initialized weights 
model_lora_v0 = get_peft_model(model, peft_config)
print("model weight", model_lora_v0.base_model.model.vision_model.transformer.layers[0].self_attn.q_proj.lora_A.default.weight)


#load the lora weights
lora_path = "/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6"
model_lora = PeftModel.from_pretrained(model_lora_v0, lora_path)
print("lora weight", model_lora.base_model.model.vision_model.transformer.layers[0].self_attn.q_proj.lora_A.default.weight)

model weight Parameter containing:
tensor([[ 0.0070, -0.0086, -0.0157,  ..., -0.0088, -0.0222, -0.0218],
        [ 0.0022,  0.0259, -0.0260,  ...,  0.0275,  0.0052,  0.0174],
        [ 0.0250, -0.0203, -0.0103,  ..., -0.0130,  0.0099,  0.0025],
        ...,
        [ 0.0035, -0.0135, -0.0140,  ...,  0.0082,  0.0201, -0.0114],
        [-0.0256,  0.0257, -0.0099,  ...,  0.0255,  0.0216,  0.0178],
        [-0.0069,  0.0078, -0.0247,  ...,  0.0108, -0.0047, -0.0061]],
       device='cuda:0', requires_grad=True)
lora weight Parameter containing:
tensor([[-0.0217,  0.0279, -0.0051,  ..., -0.0003,  0.0151, -0.0110],
        [-0.0219, -0.0182,  0.0082,  ...,  0.0265, -0.0046, -0.0235],
        [-0.0163, -0.0058, -0.0159,  ...,  0.0167,  0.0246,  0.0159],
        ...,
        [ 0.0107, -0.0135,  0.0128,  ..., -0.0223, -0.0279, -0.0178],
        [-0.0139,  0.0175,  0.0122,  ..., -0.0272, -0.0008, -0.0193],
        [-0.0176,  0.0272,  0.0188,  ...,  0.0005,  0.0259, -0.0144]],
       device='cuda

In [1]:
#load the model

import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
PYTORCH_CUDA_ALLOC_CONF=True

# Hugging Face model id
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    # attn_implementation="flash_attention_2", # not supported for training
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = AutoProcessor.from_pretrained(model_id)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Two Way of loading lora weights in model


In [5]:
# Automatically Load model with LoRA layers in pre-trained weights (saved as HF folder)

from peft import PeftModel
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

#load the pre-trained weight on model
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    # attn_implementation="flash_attention_2", # not supported for training
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

# adding lora layers in the model, with random initialized weights 
model_lora_v0 = get_peft_model(model, peft_config)
print("model weight", model_lora_v0.base_model.model.vision_model.transformer.layers[0].self_attn.q_proj.lora_A.default.weight)


#load the lora weights
lora_path = "/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6"
model_lora = PeftModel.from_pretrained(model_lora_v0, lora_path)
print("lora weight", model_lora.base_model.model.vision_model.transformer.layers[0].self_attn.q_proj.lora_A.default.weight)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

model weight Parameter containing:
tensor([[-0.0015,  0.0267, -0.0138,  ..., -0.0010, -0.0137, -0.0026],
        [ 0.0006,  0.0163,  0.0100,  ..., -0.0069,  0.0081,  0.0130],
        [ 0.0090, -0.0002, -0.0079,  ..., -0.0061,  0.0254, -0.0029],
        ...,
        [ 0.0004, -0.0190,  0.0167,  ..., -0.0216, -0.0261, -0.0077],
        [-0.0200,  0.0246,  0.0064,  ...,  0.0063, -0.0228, -0.0176],
        [ 0.0258, -0.0096, -0.0048,  ..., -0.0154,  0.0092, -0.0003]],
       device='cuda:0', requires_grad=True)
lora weight Parameter containing:
tensor([[ 0.0277,  0.0194, -0.0084,  ...,  0.0139,  0.0274, -0.0028],
        [-0.0227,  0.0144, -0.0227,  ...,  0.0059,  0.0073, -0.0254],
        [-0.0173,  0.0265, -0.0054,  ..., -0.0216,  0.0137, -0.0173],
        ...,
        [-0.0103,  0.0033, -0.0070,  ..., -0.0126, -0.0150,  0.0096],
        [-0.0044,  0.0164,  0.0109,  ..., -0.0075,  0.0279,  0.0169],
        [-0.0175, -0.0255, -0.0163,  ..., -0.0077, -0.0035,  0.0235]],
       device='cuda

In [ ]:
# Manually loading lora weights

# set LoRA layers with random initialized weights in model

from peft import LoraConfig, get_peft_model

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    # attn_implementation="flash_attention_2", # not supported for training
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)


peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)
model_lora_v1 = get_peft_model(model, peft_config)

# load the LoRA layer only weight && check the LoRA weights in model

#load the lora only layers from the file .safetensors in pretrained weights
from safetensors.torch import load_file
lora_path = "/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6"
state_dict = load_file("/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6/adapter_model.safetensors")

model_lora_v1.load_state_dict(state_dict, strict=False)

model_lora_v1.base_model.model.vision_model.transformer.layers[0].self_attn.q_proj.lora_A.default.weight


Ablation study in checking lora weights

In [40]:
#check the LoRA weights in model

from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)
model_lora_v1 = get_peft_model(model, peft_config)

from safetensors.torch import load_file
lora_path = "/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6"
state_dict = load_file("/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6/adapter_model.safetensors")
model_lora_v1.load_state_dict(state_dict, strict=False)

model_lora_v1.base_model.model.vision_model.transformer.layers[0].self_attn.q_proj.lora_A.default.weight

Parameter containing:
tensor([[-0.0150, -0.0182,  0.0008,  ..., -0.0190,  0.0199, -0.0072],
        [ 0.0119, -0.0131, -0.0270,  ..., -0.0248,  0.0155,  0.0222],
        [-0.0201,  0.0112,  0.0105,  ...,  0.0140, -0.0012, -0.0159],
        ...,
        [ 0.0081, -0.0110, -0.0112,  ..., -0.0270, -0.0160, -0.0246],
        [-0.0257, -0.0138,  0.0099,  ..., -0.0133, -0.0244,  0.0247],
        [ 0.0162, -0.0015, -0.0134,  ...,  0.0064, -0.0146, -0.0134]],
       device='cuda:0', requires_grad=True)

In [15]:
#check whether the .pth file in the HF pre-trained folder contains lora weights, answer is no

lora_path = "/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6/rng_state.pth"

lora_state_dict = torch.load(lora_path)


/tmp/ipykernel_2225826/3392726250.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_state_dict = torch.load(lora_path)


SFTConfig(output_dir='fine-tuned-visionllama', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=8, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=0.0002, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=0.3, num_train_epochs=3, max_steps=-1, lr_scheduler_type=<SchedulerType.CONSTANT: 'constant'>, lr_scheduler_kwargs={}, warmup_ratio=0.03, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='fine-tuned-visionllama/runs/Oct31_11-11-37_srv04141', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=5, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.EPOCH: 'epoch'>, sav

In [17]:
#check whether the .safetensors file in the HF pre-trained weights folder contains lora weights. Answer is yes

from safetensors.torch import load_file
lora_path = "/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6"
state_dict = load_file("/home/sa5u24/VQA/fine-tuned-visionllama/checkpoint-6/adapter_model.safetensors")
state_dict

{'base_model.model.language_model.model.layers.0.self_attn.q_proj.lora_A.weight': tensor([[ 0.0027, -0.0126,  0.0117,  ...,  0.0038,  0.0011,  0.0006],
         [ 0.0014, -0.0159, -0.0051,  ...,  0.0075,  0.0008,  0.0107],
         [-0.0120, -0.0142, -0.0112,  ..., -0.0149,  0.0006,  0.0043],
         ...,
         [ 0.0160,  0.0040, -0.0097,  ..., -0.0005,  0.0143,  0.0133],
         [-0.0125, -0.0046,  0.0108,  ...,  0.0125, -0.0070,  0.0074],
         [-0.0020, -0.0101, -0.0060,  ...,  0.0028,  0.0025,  0.0117]]),
 'base_model.model.language_model.model.layers.0.self_attn.q_proj.lora_B.weight': tensor([[ 0.0012,  0.0012,  0.0012,  ..., -0.0004, -0.0011,  0.0005],
         [ 0.0012,  0.0012,  0.0012,  ..., -0.0007, -0.0011,  0.0002],
         [ 0.0012,  0.0012,  0.0012,  ..., -0.0012, -0.0002,  0.0012],
         ...,
         [-0.0010, -0.0010, -0.0010,  ...,  0.0011,  0.0007, -0.0010],
         [-0.0010, -0.0010, -0.0010,  ...,  0.0011,  0.0004, -0.0010],
         [-0.0009, -0.0010,